# Designing a metric to compare baskets of items

The goal of this notebook is to create a metric comparing two unordered sets of items. It shall be based on structured data provided by the e-commerce platform, as well as basic natural language processing on product names.




## Loading CSV files

In [ ]:
import csv
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import scipy.optimize
import seaborn as sns
from keras.layers import Dense, Activation, Input, Embedding, Flatten
from keras.models import load_model, Model, Sequential
from keras.utils import Sequence

In [ ]:
from google.colab import drive # Il faut pouvoir lire les fichiers CSV du Drive
drive.mount('/content/drive')

Mounted at /content/drive


## Creating a dictionary with items as keys and available information on products as values

In [ ]:
filename1 = "/content/drive/MyDrive/PSC Recommandation séquentielle/Données/DataTables/codtoID.csv"
filename2 = "/content/drive/MyDrive/PSC Recommandation séquentielle/Données/Caractéristiques des produits : CodeBarre, Nom, SousClasse, Classe, Rayon, Marque/items.csv"

d1 = {}
d2 = {}
d2[0] = ['Vide'] * 6
def fill_d2():
  with open(filename1, newline='') as csvfile:
    with open(filename2, newline='') as csvfile2:
        spamreader = csv.reader(csvfile)
        spamreader2 = csv.reader(csvfile2)
        for row in spamreader:
          d1[row[0]] = row[1] # cela correspond à temporairement d[codebarre] = id
        l = d1.keys()
        for row2 in spamreader2:
          if row2[0] in l:
            d2[int(d1[row2[0]])] = row2

fill_d2()
def infos(id):
  return d2[id]

In [ ]:
def codebarre(id):
  return d2[id][0]

def nom(id):
  return d2[id][1]

def sous_classe(id):
  return d2[id][2]

def classe(id):
  return d2[id][3]

def rayon(id):
  return d2[id][4]

def marque(id):
  return d2[id][5]

## Auxiliary functions on baskets and product classification

In [ ]:
def toText(panier): # un panier est une liste d'id (int)
  l = []
  for x in panier:
    l.append(d2[x][1])
  return l

In [ ]:
par_rayons_classes_sous_classes = {}
def fillRayonsClassesSousClasses():
  for id, l in d2.items():
    if l[4] in par_rayons_classes_sous_classes.keys():
      if l[3] in par_rayons_classes_sous_classes[l[4]].keys():
        if l[2] in par_rayons_classes_sous_classes[l[4]][l[3]].keys():
          par_rayons_classes_sous_classes[l[4]][l[3]][l[2]].append(id)
        else:
          par_rayons_classes_sous_classes[l[4]][l[3]][l[2]] = [id]
      else:
        par_rayons_classes_sous_classes[l[4]][l[3]] = {l[2]: [id]}
    else:
      par_rayons_classes_sous_classes[l[4]] = {l[3]: {l[2]: [id]}}

fillRayonsClassesSousClasses()

In [ ]:
def liste_rayons(): # C'est la liste des rayons
  return list(par_rayons_classes_sous_classes.keys())

def liste_classes(rayon): # Cela donne la liste des classes d'un rayon
  return list(par_rayons_classes_sous_classes[rayon].keys())

def liste_sous_classes(rayon, classe): # Cela donne la liste des sous-classes d'une classe d'un rayon
  return list(par_rayons_classes_sous_classes[rayon][classe].keys())

def liste_produits(rayon, classe, sous_classe): # Cela donne la liste des id d'une sous-classe d'une classe d'un rayon
  return par_rayons_classes_sous_classes[rayon][classe][sous_classe]

## Leveraging item embeddings

In [ ]:
model_embedding_64 = load_model("/content/drive/MyDrive/PSC Recommandation séquentielle/Modèles/RNN/RNN_3_save") # On charge l'embedding pré-entrainé
weights_64 = model_embedding_64.get_weights()[0]
n_products = len(weights_64)

In [ ]:
def dist_embeddings(i, j):
  return np.linalg.norm(weights_64[i] - weights_64[j])

In [ ]:
def avg_dist_sous_classe(r1, c1, sc1, r2, c2, sc2): # r rayon, c classe, sc sous classe
  l1 = liste_produits(r1, c1, sc1)
  l2 = liste_produits(r2, c2, sc2)
  assert(len(l1) > 0 and len(l2) > 0)
  s = 0
  for id1 in l1:
    for id2 in l2:
      s += dist_embeddings(id1, id2)
  return s / (len(l1) * len(l2))

## Optimization function taking as input a $\verb|numpy array|$ $g$ representing a complete weighted bipartite $n \times n$ graph, and returning a perfect matching of maximum weight

In [ ]:
def optimize_hungarian(g):
  row_ind, col_ind = scipy.optimize.linear_sum_assignment(g, True) # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html
  return col_ind, g[row_ind, col_ind].sum()

## Example with a simple similarity function

In [ ]:
def sim_simple(i, j): # fonction très simple, juste pour l'exemple
  if (i == j): return 1
  if sous_classe(i) == sous_classe(j): return .9
  if classe(i) == classe(j): return .7
  if rayon(i) == rayon(j): return .5
  return 0 # rien n'est similaire, on ne regarde pas la marque ici

In [ ]:
def comparer(seq1, seq2, b = False):
  n = len(seq1)
  g = np.array([[sim_simple(seq1[i], seq2[j]) for j in range (n)] for i in range (n)])
  l, sim = optimize_hungarian(g) # l correspond à une liste [l_0, ..., l_{n-1}] telle que pour tout i, l'item seq1[i] est affecté à l'item seq2[l[i]]. Et sim est le score global de similarité, entre 0 et n. Plus sim est proche de n, plus les deux séquences sont proches
  if b: # b vaut True lorsque l'on veut afficher l'association bijective entre les deux séquences
    print(toText(seq1))
    print(toText(seq2))
    for i in range (n):
      print(nom(seq1[i]), " <-> ", nom(seq2[l[i]]), " sim = ", g[i][l[i]])
  return sim / n

In [ ]:
comparer([2, 122, 152, 59, 45, 1, 89, 9, 321, 22, 125, 4, 3, 17, 257, 5], [8979, 8, 4, 883, 7574, 13, 59, 427, 5, 2514, 2085, 1137, 157, 172, 2505, 1], True)

## Designing a similarity function between products

Here is the chosen example used as a baseline for a similarity function between products $i_1$ et $i_2$:

\\
$$
sim(i_1,i_2)= \left\{
    \begin{array}{ll}
       1 & \mbox{if } i_1 \text{ et } i_2\text{ have same barcode, }  \\
       r_{nom}(1+sim_{noms}) & \mbox{if } i_1 \text{ et } i_2  \text{ have different brands,}\\
       r_{cat}(1+ sim_{noms}) & \mbox{if } i_1 \text{ et } i_2  \text{ have different classes,}\\
       r_{ray}(1+sim_{noms}) & \mbox{if } i_1 \text{ et } i_2  \text{ are in the same area.}
    \end{array}
\right.
$$
 \\
 with $sim_{noms}=\min \left( \frac{\text{size of the longest common subsequence}}{\text{min of sizes of sequences}}p, 1 \right)$, where $p$ is a precision factor.


In [ ]:
r_nom=0.5
r_ray=0.1
r_cat=0.20
p=3/2           #si la sous séquence commune est de taille supérieure à la 2/3 de la plus petite des deux séquences, on renvoie 1

In [ ]:
def simhard(i1, i2): # i1 et i2 correspondent au même produit (=0 si c'est un non-achat)
  t1 = infos(i1)
  t2 = infos(i2)
  if (t1[0] == t2[0]): # alors ce sont les mêmes produits
    return 1
  if i1 == 0 or i2 == 0: # cas à la con
    return 0
  if t1[4] != t2[4]: # rayons différents
    return 0
  if t1[1] == t2[1]: #rayons identiques puisque sinon ça aurait déjà retourné 0 et même sous-classe
      return (r_nom*(1+sim_noms(t1[1],t2[1],p)))
  if t1[3]==t2[3]: #rayons identiques, même catégorie, mais sous classes différentes
      return (r_cat*(1+sim_noms(t1[1],t2[1],p)))
  return  (r_ray*(1+sim_noms(t1[1],t2[1],p))) # seulement le rayon est identique

#### Similarity between strings

The following function is a dynamic programming with memoization implementation of a $O(mn)$ algorithm to compute the longest common subsequence between two input strings.

In [ ]:
def LCSubStr(s1,s2,m,n):
  #on cree le tableau qui stocke les LCSuff
  LCStuff=np.zeros([m,n])
  result=0
  for i in range(m):
    for j in range(n):
      #la ligne 0 n'a pas de sens dans notre algo
      if i==0 or j==0 :
        LCStuff[i][j]=0
      elif s1[i-1]==s2[j-1]:
        LCStuff[i][j]=LCStuff[i-1][j-1]+1
        if result<LCStuff[i][j]:
          result=LCStuff[i][j]
      else :
        LCStuff[i][j]=0
  return (result)

def LCS(s1,s2):
  return LCSubStr(s1,s2,len(s1),len(s2))

def sim_noms(s1,s2,p):
  return np.min([LCS(s1,s2)/np.min([len(s1),len(s2)])*p,1])

## Designing a similarity function between baskets of products

In [ ]:
def csv_to_table(file):
  paniers=[]
  with open(folder_res + file , "r") as csvfile:
    r = csv.reader(csvfile)
    T_ref=[]
    for row in r:
      if r.line_num%2!=0:
        T_ref = [int(i) for i in row]
      else:
        T_inf =  [int(i) for i in row]
        paniers.append([T_ref,T_inf])
 #print (paniers)
  return(paniers)


### Parameter initialization

This section handles the initialization of hyperparameters to have a decent similarity between randomly chosen item baskets.

In [ ]:
def sigmoid(z, lmbd = 16,  t = 0.3):
  return 1/(1+np.exp(-(z-t)*lmbd))

def filtre(z,lmbd=10, t=0.2):
  if z<t:
    return 0
  else:
    return (1-np.exp(-lmbd*(z-t)))/(1-np.exp(-lmbd*(1-t)))

def afficherfiltre():
  X=np.linspace(0,1,1000)
  Y=[filtre(x) for x in X]
  plt.plot(X,Y)

def comparerhard(seq1, seq2, b = False):
  n = len(seq1)
  g = np.array([[simhard(seq1[i], seq2[j]) for j in range (n)] for i in range (n)])
  l, sim = optimize_hungarian(g) # l correspond à une liste [l_0, ..., l_{n-1}] telle que pour tout i, l'item seq1[i] est affecté à l'item seq2[l[i]]. Et sim est le score global de similarité, entre 0 et n. Plus sim est proche de n, plus les deux séquences sont proches
  if b: # b vaut True lorsque l'on veut afficher l'association bijective entre les deux séquences
    print(toText(seq1))
    print(toText(seq2))
    for i in range (n):
      print(nom(seq1[i]), " <-> ", nom(seq2[l[i]]), " sim = ", g[i][l[i]])
  return filtre(sim / n)

afficherfiltre()

In [ ]:
similarities=[]
for i in range (1000):
  panier1=np.random.randint(14370, size=16)
  panier2=np.random.randint(14370, size=16)
  similarities.append(comparerhard(panier1,panier2,False))
print('Minimum des similarités:')
print (np.min(similarities))
print('Maximum des similarités:')
print (np.max(similarities))
print('Moyenne des similarités:')
print (np.mean(similarities))
ax=sns.displot(data=similarities, kde=True, bins=25, color='black')
ax.set(xlabel="similarité",ylabel="occurrences", title="Distribution des similarités pour une centaine de bi paniers prédits avec k=10")
plt.show()

## Evaluating models

### Influence of the beam size $k$ on evaluation metrics

In [ ]:
def affichercourbesk(K=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,25,30,35,40,45,50,60,70],limit=1000):
  #K=np.arange(1,21)
  means=[]
  firstquart=[]
  dec40=[]
  dec60=[]
  lastquart=[]
  meds=[]
  #fig, axs = plt.subplots((len(K))//4+1, 4,squeeze=False)
  #plt.rcParams["figure.figsize"] = (20, 20)
  #j=0
  for k in K:
    #j+=1
    file_res="Y_dev_k"+str(k)+".csv"
    paniers=csv_to_table(file_res)
    sim=[]
    for i in range(np.min([len(paniers),limit])):
      sim.append(comparerhard(paniers[i][0], paniers[i][1],False))
    print('--------------------------')
    print('--------------------------')
    print('RESULTAT POUR k='+str(k)+":")
    print('--------------------------')
    print('Minimum des similarités:')
    print (np.min(sim))
    print('Maximum des similarités:')
    print (np.max(sim))
    print('Moyenne des similarités:')
    mean=np.mean(sim)

    firstq=np.percentile(sim, 25)
    dec4=np.percentile(sim,40)
    med=np.percentile(sim,50)
    dec6=np.percentile(sim,60)
    lastq=np.percentile(sim,75)
    firstquart.append(firstq)
    means.append(mean)
    dec40.append(dec4)
    meds.append(med)
    dec60.append(dec6)
    lastquart.append(lastq)

    print (mean)
    print(' ')
    #Si l'on veut afficher toutes les distributions:
    #graph=sns.histplot(data=sim, kde=True, color='grey', bins=20,ax=axs[(j-1)//4, (j-1)%4])
    #graph.set(xlabel="similarité",ylabel="occurrences",title='Distribution pour k=' +str(k))
  plt.plot(K,means, color="blue")
  plt.plot(K,firstquart, color='purple')
  plt.plot(K, lastquart, color='yellow')
  plt.plot(K,dec40, color='orange')
  plt.plot(K,meds, color='gray')
  plt.plot(K,dec60, color="red")
  plt.show()


affichercourbesk()

### Evaluating on development sets

In [ ]:
def comparaison_dev_test(K=[1,5,10,25],limit=1000):
  means_dev=[]
  means_test=[]
  plt.rcParams["figure.figsize"] = (10, 10)

  for k in K:
    #notons où nous allons chercher les données
    file_res_dev="Y_dev_k"+str(k)+".csv"
    file_res_test="Y_test_k"+str(k)+".csv"
    paniers_dev=csv_to_table(file_res_dev)
    paniers_test=csv_to_table(file_res_test)

    #créons nos deu
    sim_dev=[]
    sim_test=[]
    for i in range(np.min([len(paniers_dev),limit])):
      sim_dev.append(comparerhard(paniers_dev[i][0], paniers_dev[i][1],False))
    for i in range(np.min([len(paniers_test),limit])):
      sim_test.append(comparerhard(paniers_test[i][0], paniers_test[i][1],False))

    #enregistrons les valeurs moyennes
    mean_dev=np.mean(sim_dev)
    mean_test=np.mean(sim_test)
    means_dev.append(mean_dev)
    means_test.append(mean_test)
  plt.plot(K,means_dev, color="blue")
  plt.plot(K,means_test, color="green")
  plt.show()

comparaison_dev_test()

In [ ]:
def distribCNN(limit=1000):
  file_res="Y_dev_CNN.csv"
  paniers=csv_to_table(file_res)
  sim=[]
  for i in range(np.min([len(paniers),limit])):
    sim.append(comparerhard(paniers[i][0], paniers[i][1],False))
  print('--------------------------')
  print('--------------------------')
  print('RESULTATS CNN dev')
  print('--------------------------')
  print('Minimum des similarités:')
  print (np.min(sim))
  print('Maximum des similarités:')
  print (np.max(sim))
  print('Moyenne des similarités:')
  print (np.mean(sim))
  print(' ')
  plt.figure(figsize=(10,10))
  graph=sns.histplot(data=sim, kde=True, color='green', bins=40)
  graph.set(xlabel="similarité",ylabel="occurrences",title="Resultat pour CNN")


  file_res="Y_test_CNN.csv"
  paniers=csv_to_table(file_res)
  sim=[]
  for i in range(np.min([len(paniers),limit])):
    sim.append(comparerhard(paniers[i][0], paniers[i][1],False))
  print('--------------------------')
  print('--------------------------')
  print('RESULTATS CNN test')
  print('--------------------------')
  print('Minimum des similarités:')
  print (np.min(sim))
  print('Maximum des similarités:')
  print (np.max(sim))
  print('Moyenne des similarités:')
  print (np.mean(sim))
  print(' ')
  graph=sns.histplot(data=sim, kde=True, color='blue', bins=40)
  graph.set(xlabel="similarité",ylabel="occurrences",title="Resultat pour CNN")
  plt.show()